# *eda-3-composite-plots.ipynb*

# Processing Reflectance data

Results are saved (in folder data/figures).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import rasterio
from scipy.interpolate import CubicSpline
import sys
sys.path.append('../src/')
import aposch_composite_interpolate as aci
scheme_5day, scheme_14day = aci.get_period_schemes()

## Set parameters here

In [2]:
## SET PARAMETERS
tile='15TVG'
year=2018
period_scheme=scheme_14day
scheme_name='14day'

## Composite to periods

This takes about an hour on August's laptop.

In [ ]:
composited_tile_year = aci.composite_tile_year(tile=tile,
                   year=year,
                   period_scheme=period_scheme)

features, feature_names = composited_tile_year

Finished period ending DOY 90.
Finished period ending DOY 104.
A band file could not be read: ../data/hls_23feb23/L30/2018/15/T/V/G/HLS.L30.T15TVG.2018106T165256.v2.0/HLS.L30.T15TVG.2018106T165256.v2.0.B06.tif
Marking that as bad data and continuing as normal with the rest of the files.
Finished period ending DOY 118.
Finished period ending DOY 132.
Finished period ending DOY 146.
Finished period ending DOY 160.
Finished period ending DOY 174.
Finished period ending DOY 188.
Finished period ending DOY 202.
Finished period ending DOY 216.
Finished period ending DOY 230.
Finished period ending DOY 244.
Finished period ending DOY 258.
Finished period ending DOY 272.
Finished period ending DOY 286.


## Reorganize features into a numpy array

In [ ]:
visualize_missing_data(features, tile, year, scheme_name,
                       save_loc = '../figures/eda-missing'):
    print(f'Creating missing data visuals for {tile}_{year}_{scheme_name}...')

In [ ]:
n_pixels = len(features[0])
n_features = len(features)    

In [ ]:
missing_mask = np.empty(shape=(n_pixels,n_features),
                        dtype='bool_')

for j in range(n_features):
    missing_mask[:,j] = np.array(features[j])==-9999

## Make one visual (heatmap)

In [6]:
pixelwise = missing_mask.sum(axis=1)
# this is the number of missing features by pixel

for_heatmap = pixelwise.reshape((3660,3660))

In [7]:
file_name = f'Heatmap_Missing_Refl_{tile}_{year}_{scheme_name}.png'

plt.figure(figsize=(20, 20))
plt.title(f'Heatmap: Missing features for Refl_{tile}_{year}_{scheme_name}',
          size=28)
plt.imshow(for_heatmap, cmap='OrRd',vmax=n_features)
plt.colorbar()

plt.savefig(f'{save_loc}/{file_name}', bbox_inches='tight')
plt.close()

## Make 2nd visual (histogram)

In [8]:
file_name = f'Histogram_Missing_Refl_{tile}_{year}_{scheme_name}.png'

plt.rcParams.update({'font.size': 24})
plt.figure(figsize=(15,15))
plt.hist(pixelwise, bins=range(0,n_features//2,6), color=(1,0.4,0.1))
plt.title(f'Histogram of Missing Features for Pixels in Refl_{tile}_{year}_{scheme_name}',
          size=28)
plt.ylabel('Nbr of pixels')
plt.xlabel('Nbr of missing features')

plt.savefig(f'{save_loc}/{file_name}', bbox_inches='tight')
plt.close()

## Make 3rd visual (bar chart)

In [9]:
period_names = []
period_values = []

for period_id, period_tuple in enumerate(period_scheme):
    period_names.append(f'{period_tuple[0]}-{period_tuple[1]}')
    
    slice_start = period_id * 6
    slice_end = period_id * 6 + 6
    pixelbands = 6 * n_pixels
    value = missing_mask[:,slice_start:slice_end].sum() / pixelbands
    
    period_values.append(value)

In [10]:
file_name = f'Bar_Chart_Missing_Refl_{tile}_{year}_{scheme_name}.png'

plt.figure(figsize=(20,10))
plt.rcParams.update({'font.size': 20})
plt.bar(period_names, period_values, color=(1,0.2,0.6))
plt.title(f'Missing Data by Period for Refl_{tile}_{year}_{scheme_name}',
          size=24)
plt.ylabel('Percent missing pixel-bands')
plt.xlabel('Period (DOY)')
plt.xticks(rotation=45)

plt.savefig(f'{save_loc}/{file_name}', bbox_inches='tight')
plt.close()

print(f'Finished missing data visuals for {tile}_{year}_{scheme_name}.')